In [1]:
import os
import csv
import numpy as np
from numpy import save
from numpy import load
from os import path
import os
from bisect import bisect
import sys
import json
import pprint
import random
import math

In [2]:
path_to_files = '../precalculated_files_mel/mel_mid/a'
output_dir = 'image_pairs/good3'
files = list(os.listdir(path_to_files))

In [3]:
def is_int(x):
    try: 
        float(x)
        return True
    except ValueError:
        return False
    
def resolve_genres(genres):
    if(len(genres) == 0): return mapping[genres[0].strip()]
    
    mapped = []
    for genre in genres:
        genre = genre.strip()
        if(genre in mapping): mapped.append(mapping[genre])
        
    unique = set(mapped)
    return list(unique)

def load_map(filename):
    with open(filename) as f:
        return json.loads(f.read())
    
def save_map(m, filename):
    with open(filename, 'w') as f:
        f.write(json.dumps(m))

Creating positive pairs

In [ ]:
song_count = 16000

total_x = np.zeros(shape=(song_count*2,2,217,334),dtype=np.float16)
total_y = np.zeros(shape=(song_count*2),dtype=np.uint8)

limit = 2000
step_count = 6
counter = 0
for key in genres_holder.keys():
    
    list_of_names = genres_holder[key]
    np.flip(list_of_names)
    added = counter

    for step in range(step_count-3):

        for i, name in enumerate(list_of_names[step::step_count]):
            next_name = list_of_names[i+1]
            if(not path.isfile(path_to_files+'/'+name+'.png') or 
               not path.isfile(path_to_files+'/'+next_name+'.png')): continue
            
            img, img2 = cv2.imread(path_to_files+'/'+name+'.png'), cv2.imread(path_to_files+'/'+next_name+'.png')
            gray, gray2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
            total_x[counter] = [gray/255,gray2/255]
            total_y[counter] = 1
            
            counter += 1
            if(counter-added >= limit):break
        if(counter-added >= limit):
            break

Creating the negative pairsy

In [13]:
genres = list(genres_holder.keys())
pair_count = 0
while(pair_count < song_count):
        
    g1 = random.choice(genres)
    g2 = random.choice(genres)
    if(g1 == g2): continue
        
    name1 = None
    while(True):
        name1 = random.choice(genres_holder[g1])
        if(path.isfile(path_to_files+'/'+name1+'.png')): break
        
    name2 = None
    while(True):
        name2 = random.choice(genres_holder[g2])
        if(path.isfile(path_to_files+'/'+name2+'.png')): break

    img, img2 = cv2.imread(path_to_files+'/'+name1+'.png'), cv2.imread(path_to_files+'/'+name2+'.png')
    gray, gray2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    
    total_x[song_count+pair_count] = [gray/255,gray2/255]
    total_y[song_count+pair_count] = 0
    
    sys.stdout.write("%s\t%d\r" % (pair_count, song_count)  )
    sys.stdout.flush()
    pair_count += 1
